In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics import pairwise_distances

In [2]:
train_df=pd.read_csv('/home/subset_imdb19.csv')
train_df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


The first column contains the reviews as a string. The second contains the class, which we will not use.

Read the first review.

In [3]:
review_id = 0
review = train_df.iloc[review_id].review
review

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

As you can see, this contains HTML tags and punctuations. For your convenience, we preprocessed the dataset beforehand.

In [4]:
with open('/home/dataset_file_0_19.dat', 'r') as file:
    dataset = file.readlines()
processed = dataset[review_id]    
processed

'one of the other reviewers has mentioned that after watching just 1 oz episode you ll be hooked they are right as this is exactly what happened with me the first thing that struck me about oz was its brutality and unflinching scenes of violence which set in right from the word go trust me this is not a show for the faint hearted or timid this show pulls no punches with regards to drugs sex or violence its is hardcore in the classic use of the word it is called oz as that is the nickname given to the oswald maximum security state penitentary it focuses mainly on emerald city an experimental section of the prison where all the cells have glass fronts and face inwards so privacy is not high on the agenda em city is home to many aryans muslims gangstas latinos christians italians irish and more so scuffles death stares dodgy dealings and shady agreements are never far away i would say the main appeal of the show is due to the fact that it goes where other shows wouldn t dare forget pretty

Note that we removed punctuation signs, leaving only lowercase words separated by spaces.

Can you still understand the content of the text?

### **⏸ How many words does the first review have?**

#### A. 314
#### B. 256
#### C. 160

In [5]:
### edTest(test_chow1) ###
#your answer here
answer1 = 'A'

The final ingredient are the ELMo embeddings. 
The embeddings are contained in a dictionary, where each key is the review number and the value a numpy array containing the corresponding embedding.

We will use the embeddings of the fist review.

In [6]:
with open('/home/elmo_embeddings_0_19.pkl', 'rb') as file:
    elmo_dict = pickle.load(file)
sentence_embeddings = elmo_dict[review_id]    

### **⏸ Which dimension corresponds to the number of words?**

#### A. 0
#### B. 1
#### C. 2


In [7]:
### edTest(test_chow2) ###
answer2 = 'B'

Now, lets study some of the word embeddings. 

In [8]:
# Split the review by ' ', transform it into a numpy array 
#and find the indices that  correspond to the word prison.
index = np.array(dataset[review_id].split(' '))=='prison'
# Filter the embeddings of the word prison
sel_elmo = sentence_embeddings[:,index,:]

Your should have three embeddings, corresponding to the number of times the word appears in the review.

Compare values of the dimensions for the same word.

### **⏸ In which dimension they have values in common?**

#### A. 0
#### B. 1
#### C. 2



In [9]:
### edTest(test_chow3) ###
answer3 = 'A'

Create your own secret sauce. Put emphasis on the higher order embeddings as shown below.

In [10]:
def secret_sauce(embeddings, s):
    s = np.array(s)
    return s[0]*embeddings[0,:,:]+s[1]*embeddings[1,:,:]+s[2]*embeddings[2,:,:]


In [11]:
sauce = secret_sauce(sentence_embeddings, [0.1, 0.4, 0.5])

Compute the cosine similarity.

In [12]:
# We compute the cosine similarity between all the embeddings in sauce
dist = pairwise_distances(sauce, metric='cosine', n_jobs=-1)

# We are looking for the closest one, but the diagonal will always be 0. 
# We change the diagonal to 999 (a big number)
dist[np.arange(dist.shape[0]), np.arange(dist.shape[0])] = 999

# We find the elements corresponding to the words we selected.
# We use argsort to find the nearest neighbors
neighbors = dist[index,:].argsort(axis=1)

And now we can get the closest words acording to our sauce embeddings. 

In [13]:
review = np.array(dataset[review_id].split(' '))
for i in range(neighbors.shape[0]):
    print(review[neighbors[i,:5]])

['state' 'city' 'security' 'prison' 'penitentary']
['prison' 'bitches' 'prison' 'inmates' 'crooked']
['prison' 'street' 'experience' 'skills' 'prison']


You can see that even with the same word, the neighbors are different, with different meaning each time. 
This is because we put more emphasis on the higher level embeddings.

Compute the sauce for $s = [1,0,0]$.

In [14]:
# We change s to [1,0,0] 
sauce = secret_sauce(sentence_embeddings, [1, 0, 0])

# Complete here